In [55]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

from pandas.testing import assert_frame_equal
from lightgbm import LGBMRegressor

import lightgbm as lgb


In [3]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [4]:
train = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Retail3\train.csv')
test = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Retail3\test.csv')
submission = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Retail3\sample_submission.csv')

In [5]:
print('\n******* title ********\n', 'shape: ', train.shape, '\n', train.columns, '\n', train.describe(), '\n', train.isnull().sum() )

print('\n******* title ********\n', 'shape: ', test.shape, '\n', test.columns, '\n', test.describe(), '\n', test.isnull().sum() )


print('\n******* title ********\n', 'shape: ', submission.shape, '\n', submission.columns, '\n', submission.describe(), '\n', submission.isnull().sum() )



******* title ********
 shape:  (913000, 4) 
 Index(['date', 'store', 'item', 'sales'], dtype='object') 
           store      item     sales
count 913000.00 913000.00 913000.00
mean       5.50     25.50     52.25
std        2.87     14.43     28.80
min        1.00      1.00      0.00
25%        3.00     13.00     30.00
50%        5.50     25.50     47.00
75%        8.00     38.00     70.00
max       10.00     50.00    231.00 
 date     0
store    0
item     0
sales    0
dtype: int64

******* title ********
 shape:  (45000, 4) 
 Index(['id', 'date', 'store', 'item'], dtype='object') 
             id    store     item
count 45000.00 45000.00 45000.00
mean  22499.50     5.50    25.50
std   12990.53     2.87    14.43
min       0.00     1.00     1.00
25%   11249.75     3.00    13.00
50%   22499.50     5.50    25.50
75%   33749.25     8.00    38.00
max   44999.00    10.00    50.00 
 id       0
date     0
store    0
item     0
dtype: int64

******* title ********
 shape:  (45000, 2) 
 Index

In [6]:
test_shop_ids = test['store'].unique()
test_item_ids = test['item'].unique()

lk_train = train[train['store'].isin(test_shop_ids)]

lk_train = train[train['item'].isin(test_item_ids)]







In [7]:
lk_train.equals(train)

True

In [8]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [9]:
train['date'] = pd.to_datetime(train.date,format="%Y-%m-%d")

train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

train.head()

,date,store,item,sales,month,day
0,2013-01-01,1,1,13,1,1
1,2013-01-02,1,1,11,1,2
2,2013-01-03,1,1,14,1,3
3,2013-01-04,1,1,13,1,4
4,2013-01-05,1,1,10,1,5


In [10]:
train = train.sort_values('date')


In [11]:
train2 = train.copy()

train2 = train2.groupby(by=['month', 'day', 'store', 'item'], as_index=False).agg({'sales':['mean', 'min', 'max', 'std']})

train2.head()

month day store item sales             
                        mean min max  std
0     1   1     1    1 14.60   9  21 5.18
1     1   1     1    2 34.80  32  40 3.27
2     1   1     1    3 22.60  15  36 8.08
3     1   1     1    4 14.00   9  21 4.80
4     1   1     1    5 14.00   6  25 6.96

In [12]:
train2.columns = ['month', 'day', 'store', 'item', 'mean_sales', 'min_sales', 'max_sales', 'std_sales']

train2.head()


,month,day,store,item,mean_sales,min_sales,max_sales,std_sales
0,1,1,1,1,14.60,9,21,5.18
1,1,1,1,2,34.80,32,40,3.27
2,1,1,1,3,22.60,15,36,8.08
3,1,1,1,4,14.00,9,21,4.80
4,1,1,1,5,14.00,6,25,6.96


In [13]:
train2.shape

(183000, 8)

In [14]:
train2.describe()

,month,day,store,item,mean_sales,min_sales,max_sales,std_sales
count,183000.00,183000.00,183000.00,183000.00,183000.00,183000.00,183000.00,182500.00
mean,6.51,15.76,5.50,25.50,52.21,38.86,66.31,11.14
std,3.45,8.81,2.87,14.43,26.47,21.42,32.88,6.08
min,1.00,1.00,1.00,1.00,4.20,0.00,5.00,0.00
25%,4.00,8.00,3.00,13.00,30.60,22.00,40.00,6.58
50%,7.00,16.00,5.50,25.50,48.00,35.00,61.00,9.91
75%,10.00,23.00,8.00,38.00,69.40,52.00,87.00,14.46
max,12.00,31.00,10.00,50.00,160.80,134.00,231.00,46.73


# empty df

In [15]:
store_ids = train2['store'].unique()
item_ids = train2['item'].unique()
empty_df = []
for i in range (1,13):
    for j in range(1,32):
        for store in store_ids:
            for item in item_ids:
                empty_df.append([i, j, store, item])


    
empty_df = pd.DataFrame(empty_df, columns=['month', 'day','store','item'])

In [16]:
train2 = pd.merge(empty_df, train2, on=['month', 'day','store','item'], how='left')
train.fillna(0, inplace=True)

print( train2.head() )

print( 'train2:  ', train2.shape ) # became 6 million rows

   month  day  store  item  mean_sales  min_sales  max_sales  std_sales
0      1    1      1     1       14.60       9.00      21.00       5.18
1      1    1      1     2       34.80      32.00      40.00       3.27
2      1    1      1     3       22.60      15.00      36.00       8.08
3      1    1      1     4       14.00       9.00      21.00       4.80
4      1    1      1     5       14.00       6.00      25.00       6.96
train2:   (186000, 8)


# shift

In [17]:
train3 = train2.copy()

train3['shift_sales'] =  train3.groupby(['store', 'item'])['mean_sales'].shift(-1)

print ( train3.head(5) )
print( train3.isnull().sum())


   month  day  store  item  mean_sales  min_sales  max_sales  std_sales  \
0      1    1      1     1       14.60       9.00      21.00       5.18   
1      1    1      1     2       34.80      32.00      40.00       3.27   
2      1    1      1     3       22.60      15.00      36.00       8.08   
3      1    1      1     4       14.00       9.00      21.00       4.80   
4      1    1      1     5       14.00       6.00      25.00       6.96   

   shift_sales  
0        14.40  
1        38.80  
2        24.80  
3        13.60  
4         9.60  
month             0
day               0
store             0
item              0
mean_sales     3000
min_sales      3000
max_sales      3000
std_sales      3500
shift_sales    3500
dtype: int64


In [18]:
# min_prediods= 1 =) we don't have null values ( at least
# one observation required to have a value)
# Min value 
f_min = lambda x: x.rolling(window=3, min_periods=1).min() 
# Max value
f_max = lambda x: x.rolling(window=3, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=3, min_periods=1).mean()
# Standard deviation
# std: dispersion of a dataset relative to its mean and is calculated as the square root of the variance
f_std = lambda x: x.rolling(window=3, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']
for i in range(len(function_list)):
    train3[('sales_lag_%s' % function_name[i])] = train.groupby(['store', 'item'])['sales'].apply(function_list[i])

# Fill the empty std features with 0
train3.fillna(0, inplace=True)

train3.head()

,month,day,store,item,mean_sales,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std
0,1,1,1,1,14.60,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00
1,1,1,1,2,34.80,32.00,40.00,3.27,38.80,11.00,13.00,12.00,1.41
2,1,1,1,3,22.60,15.00,36.00,8.08,24.80,11.00,14.00,12.67,1.53
3,1,1,1,4,14.00,9.00,21.00,4.80,13.60,11.00,14.00,12.67,1.53
4,1,1,1,5,14.00,6.00,25.00,6.96,9.60,10.00,14.00,12.33,2.08


In [19]:
lag_list = [1, 2, 3]

for lag in lag_list:
    ft_name = ('sales_shifted%s' % lag)
    train3[ft_name] = train3.groupby(['store' , 'item'])['mean_sales'].shift(lag)
    # Fill the empty shifted features with 0
    train3[ft_name].fillna(0, inplace=True)

train3.head()
    

,month,day,store,item,mean_sales,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std,sales_shifted1,sales_shifted2,sales_shifted3
0,1,1,1,1,14.60,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00,0.00,0.00,0.00
1,1,1,1,2,34.80,32.00,40.00,3.27,38.80,11.00,13.00,12.00,1.41,0.00,0.00,0.00
2,1,1,1,3,22.60,15.00,36.00,8.08,24.80,11.00,14.00,12.67,1.53,0.00,0.00,0.00
3,1,1,1,4,14.00,9.00,21.00,4.80,13.60,11.00,14.00,12.67,1.53,0.00,0.00,0.00
4,1,1,1,5,14.00,6.00,25.00,6.96,9.60,10.00,14.00,12.33,2.08,0.00,0.00,0.00


In [20]:
train3

,month,day,store,item,mean_sales,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std,sales_shifted1,sales_shifted2,sales_shifted3
0,1,1,1,1,14.60,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00,0.00,0.00,0.00
1,1,1,1,2,34.80,32.00,40.00,3.27,38.80,11.00,13.00,12.00,1.41,0.00,0.00,0.00
2,1,1,1,3,22.60,15.00,36.00,8.08,24.80,11.00,14.00,12.67,1.53,0.00,0.00,0.00
3,1,1,1,4,14.00,9.00,21.00,4.80,13.60,11.00,14.00,12.67,1.53,0.00,0.00,0.00
4,1,1,1,5,14.00,6.00,25.00,6.96,9.60,10.00,14.00,12.33,2.08,0.00,0.00,0.00
5,1,1,1,6,35.80,30.00,52.00,9.18,35.00,10.00,13.00,11.67,1.53,0.00,0.00,0.00
6,1,1,1,7,34.20,25.00,42.00,6.69,36.00,10.00,12.00,10.67,1.15,0.00,0.00,0.00
7,1,1,1,8,46.20,33.00,65.00,13.74,45.40,9.00,12.00,10.33,1.53,0.00,0.00,0.00
8,1,1,1,9,30.60,18.00,47.00,13.01,26.80,9.00,12.00,10.33,1.53,0.00,0.00,0.00
9,1,1,1,10,42.20,33.00,53.00,8.17,47.20,9.00,12.00,10.00,1.73,0.00,0.00,0.00


In [21]:
train3['item_trend'] = train['sales']

for lag in lag_list:
    ft_name = ('sales_shifted%s' % lag)
    train3['item_trend'] -= train3[ft_name]

train3['item_trend'] /= len(lag_list) + 1

train3.head()


,month,day,store,item,mean_sales,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std,sales_shifted1,sales_shifted2,sales_shifted3,item_trend
0,1,1,1,1,14.60,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00,0.00,0.00,0.00,3.25
1,1,1,1,2,34.80,32.00,40.00,3.27,38.80,11.00,13.00,12.00,1.41,0.00,0.00,0.00,2.75
2,1,1,1,3,22.60,15.00,36.00,8.08,24.80,11.00,14.00,12.67,1.53,0.00,0.00,0.00,3.50
3,1,1,1,4,14.00,9.00,21.00,4.80,13.60,11.00,14.00,12.67,1.53,0.00,0.00,0.00,3.25
4,1,1,1,5,14.00,6.00,25.00,6.96,9.60,10.00,14.00,12.33,2.08,0.00,0.00,0.00,2.50


In [22]:
# Shop mean encoding
gp_store_mean = train3.groupby(['store']).agg({'mean_sales': ['mean']})
#print( gp_store_mean ) # groupby eliminate all columns but shop_id and item_cnt_month: mean
gp_store_mean.columns = ['shop_mean']
#print( gp_store_mean )
gp_store_mean.reset_index(inplace=True)
print( gp_store_mean )

train3 = pd.merge(train3, gp_store_mean, on=['store'], how='left')

train3.head()

   store  shop_mean
0      1      46.47
1      2      65.90
2      3      58.53
3      4      53.98
4      5      39.10
5      6      39.06
6      7      35.75
7      8      63.06
8      9      54.12
9     10      57.72


,month,day,store,item,mean_sales,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std,sales_shifted1,sales_shifted2,sales_shifted3,item_trend,shop_mean
0,1,1,1,1,14.60,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00,0.00,0.00,0.00,3.25,46.47
1,1,1,1,2,34.80,32.00,40.00,3.27,38.80,11.00,13.00,12.00,1.41,0.00,0.00,0.00,2.75,46.47
2,1,1,1,3,22.60,15.00,36.00,8.08,24.80,11.00,14.00,12.67,1.53,0.00,0.00,0.00,3.50,46.47
3,1,1,1,4,14.00,9.00,21.00,4.80,13.60,11.00,14.00,12.67,1.53,0.00,0.00,0.00,3.25,46.47
4,1,1,1,5,14.00,6.00,25.00,6.96,9.60,10.00,14.00,12.33,2.08,0.00,0.00,0.00,2.50,46.47


In [23]:
# Item mean encoding.
gp_item_mean = train3.groupby(['item']).agg({'mean_sales': ['mean']})
gp_item_mean.columns = ['item_mean']
gp_item_mean.reset_index(inplace=True)
train3 = pd.merge(train3, gp_item_mean, on=['item'], how='left')

# Shop with item mean encoding.
gp_shop_item_mean = train3.groupby(['store', 'item']).agg({'mean_sales': ['mean']})
gp_shop_item_mean.columns = ['store_item_mean']
gp_shop_item_mean.reset_index(inplace=True)
train3 = pd.merge(train3, gp_shop_item_mean, on=['store', 'item'], how='left')

In [24]:
# Month mean encoding.
gp_month_mean = train3.groupby(['month']).agg({'mean_sales': ['mean']})
gp_month_mean.columns = ['month_mean']
gp_month_mean.reset_index(inplace=True)
train3 = pd.merge(train3, gp_month_mean, on=['month'], how='left')

In [25]:

#train_set = train3

train_set = train3[ train3['month']<=10 ]
#print( train_set.shape )

validation_set = train3[ train3['month']>10 ]
print ( validation_set.shape)

#print( 'remember we have excluded first 3 months')


(31000, 21)


In [26]:
X_train = train_set.drop(['mean_sales'], axis=1)
Y_train = train_set['mean_sales'].astype(int)
X_validation = validation_set.drop(['mean_sales'], axis=1)
Y_validation = validation_set['mean_sales'].astype(int)

# random forest regressor

In [27]:
rf_features = X_train.columns

rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(rf_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [28]:
rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)

rmse = np.sqrt(mean_squared_error(rf_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(rf_val_pred, Y_validation))
print('rmse of validation : ' , rmse)


x = smape(rf_train_pred, Y_train)
print('smape of train : ' , x)
               
x = smape(rf_val_pred, Y_validation)
print('smape of validation : ' , x)


print( rf_model.score(rf_train, Y_train) )
print( rf_model.score(rf_val, Y_validation) )

rmse of train :  2.630182655199768
rmse of validation :  2.452214662712647
smape of train :  4.148505187350732
smape of validation :  4.399312791172606
0.9908463129069007
0.9896908244073822


# linear regression

In [29]:
lr_features = X_train.columns


lr_train = X_train[lr_features]
lr_val = X_validation[lr_features]


In [30]:
lr_scaler = MinMaxScaler()
lr_scaler.fit(lr_train)
lr_train = lr_scaler.transform(lr_train)
lr_val = lr_scaler.transform(lr_val)
lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(lr_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [31]:
lr_train_pred = lr_model.predict(lr_train)
lr_val_pred = lr_model.predict(lr_val)

rmse = np.sqrt(mean_squared_error(lr_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(lr_val_pred, Y_validation))
print('rmse of validation : ' , rmse)


print( lr_model.score(lr_train, Y_train) )
print( lr_model.score(lr_val, Y_validation) )

x = smape(rf_train_pred, Y_train)
print('smape of train : ' , x)
               
x = smape(rf_val_pred, Y_validation)
print('smape of validation : ' , x)


rmse of train :  2.6749588805373206
rmse of validation :  2.4766766215221505
0.9905319953081264
0.9894841210947526
smape of train :  4.148505187350732
smape of validation :  4.399312791172606


# lightgbm

In [32]:
lgbm_model=LGBMRegressor(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=32,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.04,
        reg_lambda=0.073,
        min_split_gain=0.0222415,
        min_child_weight=40)

In [33]:
lgbm_model.fit(X_train, Y_train)


lightgbm_train_pred = lgbm_model.predict(X_train)
lightgbm_val_pred = lgbm_model.predict(X_validation)

rmse = np.sqrt(mean_squared_error(Y_train, lightgbm_train_pred ))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(Y_validation, lightgbm_val_pred))
print('rmse of validation : ' , rmse)

x = smape(rf_train_pred, Y_train)
print('smape of train : ' , x)
               
x = smape(rf_val_pred, Y_validation)
print('smape of validation : ' , x)

print( lgbm_model.score(X_train, Y_train) )
print( lgbm_model.score(X_validation, Y_validation) )

rmse of train :  2.239597629742466
rmse of validation :  2.1331917387982147
smape of train :  4.148505187350732
smape of validation :  4.399312791172606
0.9933631157721166
0.9921987040556478


In [34]:
"""
xgb_features = X_train.columns


xgb_train = X_train[xgb_features]
xgb_val = X_validation[xgb_features]

xgb_model = XGBRegressor(max_depth=8, 
                         n_estimators=500, 
                         min_child_weight=1000,  
                         colsample_bytree=0.7, 
                         subsample=0.7, 
                         eta=0.3, 
                         seed=0)
xgb_model.fit(xgb_train, 
              Y_train, 
              eval_metric="rmse", 
              eval_set=[(xgb_train, Y_train), (xgb_val, Y_validation)], 
              verbose=20, 
              early_stopping_rounds=20)
    """

'\nxgb_features = X_train.columns\n\n\nxgb_train = X_train[xgb_features]\nxgb_val = X_validation[xgb_features]\n\nxgb_model = XGBRegressor(max_depth=8, \n                         n_estimators=500, \n                         min_child_weight=1000,  \n                         colsample_bytree=0.7, \n                         subsample=0.7, \n                         eta=0.3, \n                         seed=0)\nxgb_model.fit(xgb_train, \n              Y_train, \n              eval_metric="rmse", \n              eval_set=[(xgb_train, Y_train), (xgb_val, Y_validation)], \n              verbose=20, \n              early_stopping_rounds=20)\n    '

In [35]:
"""
xgb_train_pred = xgb_model.predict(xgb_train)
xgb_val_pred = xgb_model.predict(xgb_val)


from sklearn.metrics import mean_squared_error 

rmse = np.sqrt(mean_squared_error(xgb_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(xgb_val_pred, Y_validation))
print('rmse of validation : ' , rmse)

print( xgb_model.score(X_train, Y_train) )
print( xgb_model.score(X_validation, Y_validation) )

"""

"\nxgb_train_pred = xgb_model.predict(xgb_train)\nxgb_val_pred = xgb_model.predict(xgb_val)\n\n\nfrom sklearn.metrics import mean_squared_error \n\nrmse = np.sqrt(mean_squared_error(xgb_train_pred, Y_train))\nprint('rmse of train : ' , rmse)\n               \nrmse = np.sqrt(mean_squared_error(xgb_val_pred, Y_validation))\nprint('rmse of validation : ' , rmse)\n\nprint( xgb_model.score(X_train, Y_train) )\nprint( xgb_model.score(X_validation, Y_validation) )\n\n"

In [36]:
def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgboost.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgboost.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgboost.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgboost.DMatrix(test_X)
        model = xgboost.train(plst, xgtrain, num_rounds)
        
    return model    
    
    
dmatrix_model = XGB_regressor(train_X = X_train, train_y = Y_train, test_X = X_validation, test_y = Y_validation)


[0]	train-mae:46.3407	test-mae:40.9436
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:41.7103	test-mae:36.8442
[2]	train-mae:37.5416	test-mae:33.1532
[3]	train-mae:33.7897	test-mae:29.8309
[4]	train-mae:30.4132	test-mae:26.8426
[5]	train-mae:27.3747	test-mae:24.1539
[6]	train-mae:24.6396	test-mae:21.7346
[7]	train-mae:22.1774	test-mae:19.5557
[8]	train-mae:19.9609	test-mae:17.5952
[9]	train-mae:17.9664	test-mae:15.8313
[10]	train-mae:16.1705	test-mae:14.2422
[11]	train-mae:14.5554	test-mae:12.8146
[12]	train-mae:13.1014	test-mae:11.5306
[13]	train-mae:11.7924	test-mae:10.3736
[14]	train-mae:10.6152	test-mae:9.3348
[15]	train-mae:9.55536	test-mae:8.40043
[16]	train-mae:8.60235	test-mae:7.55809
[17]	train-mae:7.74443	test-mae:6.8029
[18]	train-mae:6.97383	test-mae:6.12497
[19]	train-mae:6.28264	test-mae:5.51431
[20]	train-mae:5.66478	test-mae:4.97024
[21]	train-mae:5.11231	test-mae

In [53]:
dmatrix_pred_train = dmatrix_model.predict(xgboost.DMatrix(X_train), ntree_limit = dmatrix_model.best_ntree_limit)
dmatrix_pred_val = dmatrix_model.predict(xgboost.DMatrix(X_validation), ntree_limit = dmatrix_model.best_ntree_limit)

rmse = np.sqrt(mean_squared_error(dmatrix_pred_train, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(dmatrix_pred_val, Y_validation))
print('rmse of validation : ' , rmse)


rmse of train :  2.2232916222043153
rmse of validation :  2.1365358610917755


# lgb Dataset

In [57]:
params = {
        'nthread': 10,
         'max_depth': 5,
#         'max_depth': 9,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'mape', # this is abs(a-e)/max(1,a)
#         'num_leaves': 39,
        'num_leaves': 64,
        'learning_rate': 0.2,
       'feature_fraction': 0.9,
#         'feature_fraction': 0.8108472661400657,
#         'bagging_fraction': 0.9837558288375402,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'lambda_l1': 3.097758978478437,
        'lambda_l2': 2.9482537987198496,
#       'lambda_l1': 0.06,
#       'lambda_l2': 0.1,
        'verbose': 1,
        'min_child_weight': 6.996211413900573,
        'min_split_gain': 0.037310344962162616,
        }
    
lgbdataset_train = lgb.Dataset(X_train,Y_train)
lgbdataset_valid = lgb.Dataset(X_validation,Y_validation)

lgbdataset_model = lgb.train(params, lgbdataset_train, 3000, valid_sets=[lgbdataset_train, lgbdataset_valid],early_stopping_rounds=50, verbose_eval=50)

lgbdataset_pred_train = lgbdataset_model.predict(X_train)
lgbdataset_pred_val = lgbdataset_model.predict(X_validation)

rmse = np.sqrt(mean_squared_error(lgbdataset_pred_train, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(lgbdataset_pred_val, Y_validation))
print('rmse of validation : ' , rmse)



Training until validation scores don't improve for 50 rounds.
[50]	training's mape: 0.0429532	valid_1's mape: 0.0495682
[100]	training's mape: 0.039778	valid_1's mape: 0.0487459
[150]	training's mape: 0.0383536	valid_1's mape: 0.0492634
Early stopping, best iteration is:
[102]	training's mape: 0.0396862	valid_1's mape: 0.0486071
rmse of train :  2.3945947454650316
rmse of validation :  2.3057148274569044


# work on test 

In [37]:
print( train3.head() )
print ( test.head() )

   month  day  store  item  mean_sales  min_sales  max_sales  std_sales  \
0      1    1      1     1       14.60       9.00      21.00       5.18   
1      1    1      1     2       34.80      32.00      40.00       3.27   
2      1    1      1     3       22.60      15.00      36.00       8.08   
3      1    1      1     4       14.00       9.00      21.00       4.80   
4      1    1      1     5       14.00       6.00      25.00       6.96   

   shift_sales  sales_lag_min     ...      sales_lag_mean  sales_lag_std  \
0        14.40          13.00     ...               13.00           0.00   
1        38.80          11.00     ...               12.00           1.41   
2        24.80          11.00     ...               12.67           1.53   
3        13.60          11.00     ...               12.67           1.53   
4         9.60          10.00     ...               12.33           2.08   

   sales_shifted1  sales_shifted2  sales_shifted3  item_trend  shop_mean  \
0            0.0

In [38]:
test['date'] = pd.to_datetime(test.date,format="%Y-%m-%d")

test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day

test.head()

,id,date,store,item,month,day
0,0,2018-01-01,1,1,1,1
1,1,2018-01-02,1,1,1,2
2,2,2018-01-03,1,1,1,3
3,3,2018-01-04,1,1,1,4
4,4,2018-01-05,1,1,1,5


In [39]:
test = test.drop(['date'], axis = 1)
test.head()

,id,store,item,month,day
0,0,1,1,1,1
1,1,1,1,1,2
2,2,1,1,1,3
3,3,1,1,1,4
4,4,1,1,1,5


In [40]:
test.head()

,id,store,item,month,day
0,0,1,1,1,1
1,1,1,1,1,2
2,2,1,1,1,3
3,3,1,1,1,4
4,4,1,1,1,5


In [41]:
test.head()


,id,store,item,month,day
0,0,1,1,1,1
1,1,1,1,1,2
2,2,1,1,1,3
3,3,1,1,1,4
4,4,1,1,1,5


In [42]:
IDs = test['id']
test = test[['month', 'day', 'store', 'item']]
print(IDs.head() )
print( test.head() )
print( test.shape )

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64
   month  day  store  item
0      1    1      1     1
1      1    2      1     1
2      1    3      1     1
3      1    4      1     1
4      1    5      1     1
(45000, 4)


In [43]:
test = test.merge(train3.drop('mean_sales', axis=1), on=['month', 'day', 'store', 'item'], how='left'    )



In [44]:
print( test.shape )
print( test.isnull().sum() )




(45000, 20)
month              0
day                0
store              0
item               0
min_sales          0
max_sales          0
std_sales          0
shift_sales        0
sales_lag_min      0
sales_lag_max      0
sales_lag_mean     0
sales_lag_std      0
sales_shifted1     0
sales_shifted2     0
sales_shifted3     0
item_trend         0
shop_mean          0
item_mean          0
store_item_mean    0
month_mean         0
dtype: int64


In [45]:
trn = pd.DataFrame( { 'x':[1,1], 'y':[1,5], 'z':[6,7] }  )
tst = pd.DataFrame( { 'x':[1,1,1], 'y':[1,3,5] } )

In [46]:
print('*** tst **** \n', tst )
print('*** trn **** \n', trn )

*** tst **** 
    x  y
0  1  1
1  1  3
2  1  5
*** trn **** 
    x  y  z
0  1  1  6
1  1  5  7


In [47]:
#l = [0,1,1]
#if l in tst:
 #   print('ok')

In [48]:
#for index, row in test.iterrows():
 #   print(row)

In [49]:
test.head()

,month,day,store,item,min_sales,max_sales,std_sales,shift_sales,sales_lag_min,sales_lag_max,sales_lag_mean,sales_lag_std,sales_shifted1,sales_shifted2,sales_shifted3,item_trend,shop_mean,item_mean,store_item_mean,month_mean
0,1,1,1,1,9.00,21.00,5.18,14.40,13.00,13.00,13.00,0.00,0.00,0.00,0.00,3.25,46.47,21.61,19.64,35.52
1,1,2,1,1,11.00,19.00,2.97,12.60,21.00,23.00,22.00,1.00,14.60,0.00,0.00,2.10,46.47,21.61,19.64,35.52
2,1,3,1,1,10.00,16.00,2.41,15.00,15.00,32.00,22.33,8.74,14.40,14.60,0.00,-3.50,46.47,21.61,19.64,35.52
3,1,4,1,1,12.00,20.00,3.16,13.40,12.00,20.00,16.00,4.00,12.60,14.40,14.60,-6.40,46.47,21.61,19.64,35.52
4,1,5,1,1,10.00,17.00,2.61,15.00,25.00,30.00,27.67,2.52,15.00,12.60,14.40,-4.25,46.47,21.61,19.64,35.52


In [50]:


test_pred = dmatrix_model.predict(xgboost.DMatrix(test), ntree_limit = dmatrix_model.best_ntree_limit)


test_pred

array([14.149017, 14.010689, 12.711826, ..., 70.47047 , 67.66739 ,
       66.11591 ], dtype=float32)

In [51]:
sub = pd.DataFrame(IDs, columns=['id'])
sub['sales'] = test_pred
sub.to_csv('submission.csv', index=False)
sub.head(10)
                

,id,sales
0,0,14.15
1,1,14.01
2,2,12.71
3,3,15.31
4,4,12.99
5,5,15.17
6,6,11.76
7,7,14.00
8,8,14.51
9,9,15.69
